In [ ]:
"""
Created on Wed Jan 24 2024

    This Python script is dedicated to the analysis and optimization of retrofitting 
    European ammonia plants for the production of low-carbon hydrogen through 
    electrolysis. The script's primary function is to optimize both the design 
    (selection and sizing of subsystems) and the hourly operations of the plant 
    to minimize the Levelized Cost of Hydrogen (LCOH) while adhering to various 
    emissions constraints (emission caps).

    In detail, the script encompasses methodologies to:
    1. Evaluate renewable energy potentials for solar PV (PV), wind onshore (WON), 
       and wind offshore (WOF) within designated suitable areas.
    2. Design and size subsystems for optimal integration with existing ammonia 
       plant infrastructure.
    3. Simulate and optimize hourly plant operations to meet production targets 
       and emissions caps.
    4. Analyze the economic impact of emissions caps on the cost-effectiveness and 
       feasibility of transitioning to electrolytic hydrogen production.

    This tool serves as a decision-support system for stakeholders considering the 
    shift to greener ammonia production methods in response to increasing environmental 
    regulations and market dynamics.

Corresponding Author: 
         S. Mingolla
         smingolla@connect.ust.hk

This script is part of the supplementary material accompanying the paper.

"""

In [ ]:
# Import libraries
import random
from gurobipy import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Define the range of hours for the simulation period
start_hour = 0 # For the start of the year, set to 0
end_hour = 8760 # For a full year, set to 8760

# Select electrolyzer type: 'ALK' for Alkaline, 'SOE' for Solid Oxide, or 'ML' for Membraneless.
electrolyzer = 'ALK'  # Change to 'SOE' or 'ML' as needed

# Define a list of emission caps to test, in units of kg CO2e/kg H2. 
epsilon_H2_list = [1]

# List of regions to test for the analysis. Each region is represented by its NUTS code.
# 'AT31', 'BE21', 'BE32', 'BG33', 'BG42', 'CH01', 'CZ04', 'DEA1', 'DEB3', 'DEE0', 'DEF0',
# 'EL51', 'ES24', 'ES42', 'ES61', 'FR10', 'FRD2', 'FRF1', 'HU21', 'HU31', 'ITH5', 'LT02',
# 'NL34', 'NL42', 'NO03', 'PL21', 'PL42', 'PL52', 'PL61', 'PL81', 'RO12', 'RO31', 'SK02',
# 'UKC1', 'UKD6', 'UKE1'
region_list = ['ITH5']

# Define a list of parameter groups for sensitivity analysis
parameters_to_test = [[]]

# Define a variable to hold the type of production for EHPS:'continuous' for continuous output, 'flexible' for flexible output
production_type = "continuous"

In [ ]:
# Import dataframes containing various parameters and metrics required for the analysis:
# `df_1_land`: Represents the cost of land at the NUTS-2 level
df_land = pd.read_csv(r"LAND_PRICE_REGION_BASED.csv", index_col = 0)

# `df_2_emissions`: Contains data on grid carbon intensity at the NUTS-1 level
df_emissions = pd.read_csv(r"EMISSIONS_GRID_COUNTRY_BASED.csv", index_col = 0)

# `df_3_CF`: Includes the solar and wind capacity factors at the NUTS-2 level
df_CF_regions = pd.read_csv(r"1y_final_dataset.csv", index_col = 0)

# `df_4_generic_parameters`: Consists of generic input parameters for cost and performance calcualtion
df_generic_parameters = pd.read_csv(r"Input_parameters_generic.csv", index_col = 0)

In [ ]:
# Financial parameters:
discount_rate = 0.0

# Performance components
# Electrolyzer    
energy_demand_electrolyzers = 0.0503
eta_EL = 1/energy_demand_electrolyzers

# H2 compressors
energy_demand_compressors = 0.002 # MWh/kg H2 compressed
eta_CP = 1/energy_demand_compressors

# Li-ion batteries (BESS)
lambda_B = 0.002/(24) # self discharging coefficient batteries
eta_c_B = 0.9274 # charging efficiency batteries
eta_d_B = 0.9274 # discharging efficiency batteries
tau_c_B = 4 # minimum number of time intervals for full charge
tau_d_B = 4 # minimum number of time intervals for full discharge

# Utility-scale solar PV
eta_PV = 0.9 # efficiency PV --> This include losses: 3% inverter, 1% transfomer HV, 1% transfomer LV, 5% rectifier
A_PV = 35000 # total area per MW installed in m2 

# Utility-scale WT
eta_WT = 0.93 # efficiency wind turbines --> This include losses: 1% transfomer HV, 1% transfomer LV, 5% rectifier
A_WT = 150000 # total area per MW installed in m2

# Grid import
eta_grid = 0.94 # --> This include losses: 1% transfomer LV, 5% rectifier
distance = 5 # km from grid connection to EHPS and from PV and WT and EHPS

In [ ]:
# Define a list of parameter groups for sensitivity analysis. leave empty for main analysis
parameters_to_test = [[]]


# Scenarios for sensitivity analysis
scenarios = ['OPT', 'PES']

# Check if the first item in parameters_to_test is an empty list
if not parameters_to_test[0]:
    # If it's an empty list, conduct the normal analysis with all values at 'AVG'
    scenarios = ['AVG']

# Loop through each combination of parameters, scenarios, epsilon_H2, and regions to analyze
for parameters in parameters_to_test:
    for scenario in scenarios:
        for epsilon_H2 in epsilon_H2_list:
            for region in region_list:
                # Convert epsilon_H2 to a string format suitable for file naming
                epsilon_H2_str = str(epsilon_H2).replace('.', '_')
                # Retrieve land price from dataframe based on region
                land_price = df_land.loc['land_price', region]
                # Extract the country code prefix from the region code
                region_prefix = region[:2]
                # Construct row label for retrieving gamma_E emission factor
                row_label = 'gamma_E_' + scenario
                gamma_E = df_emissions.loc[row_label, region_prefix]

                # Load country-specific parameters from a CSV file
                df_country_specific_parameters = pd.read_csv(f"Input_parameters_{region_prefix}.csv", index_col=0)

                # Define a function to discount a dataframe over a 27-year period
                def discount_dataframe(df, discount_rate):
                    discounted_sums = {}
                    for index, row in df.iterrows():
                        discounted_sum = 0.0
                        for year in range(27):
                            discounted_sum += row[year] / ((1 + discount_rate) ** year)
                        discounted_sums[index] = discounted_sum
                    return pd.DataFrame.from_dict(discounted_sums, orient='index', columns=['discounted_sum'])

                # Apply discounting to both generic and country-specific parameters
                df_generic_discounted = discount_dataframe(df_generic_parameters, discount_rate)
                df_country_specific_discounted = discount_dataframe(df_country_specific_parameters, discount_rate)
                # Combine discounted parameters into a single dataframe for further processing
                df_parameters_ready = pd.concat([df_generic_discounted, df_country_specific_discounted])

                # Separate cost components based on whether they are scenario-specific or not
                cost_components_with_scenario = ['c_EL', 'OM_EL', 'c_B', 'p_ETS', 'c_PV', 'c_WT', 'p_E']  # components that have the scenario suffix
                cost_components_without_scenario = ['lifetime_H2_discount', 'OM_B', 'OM_PV', 'OM_WT', 'c_CP', 'OM_CP', 'c_ST', 'T_LVAC', 'T_HVAC', 'c_HVAC_wires', 'COST_RETROFIT']  # components that don't have the scenario suffix

                results = {}

                # Determine values for cost components based on selected parameters and scenarios
                for component in cost_components_with_scenario:
                    if parameters and component in parameters:
                        value = df_parameters_ready.loc[f"{component}_{scenario}"].values[0]
                    else:
                        value = df_parameters_ready.loc[f"{component}_AVG"].values[0]
                    results[component] = value

                # Assign values for cost components that are not scenario-specific
                for component in cost_components_without_scenario:
                    value = df_parameters_ready.loc[component].values[0]
                    results[component] = value

                # Convert results to global variables for use in subsequent calculations
                for component in results.keys():
                    globals()[component] = results[component]

                # If there are no emissions, set HVAC-related costs to zero
                if epsilon_H2 == 0:
                    T_HVAC, c_HVAC_wires = 0, 0

                # Set the capital cost for the electrolyzer based on its type
                if electrolyzer == 'SOE':
                    c_EL = 6224172.67
                elif electrolyzer == 'ML':
                    c_EL = 752280.11

                # Retrieve solar and wind capacity factors for the region and time frame
                s = df_CF_regions[f'{region}PV'][start_hour:end_hour].tolist()
                w = df_CF_regions[f'{region}WT'][start_hour:end_hour].tolist()

                # Create empty dataframes for storing design and operation parameters
                df_design = pd.DataFrame()
                df_operation = pd.DataFrame()

                # Define the lifetime of the plant in years
                lifetime_plant = 26

                # Determine the number of time-steps from the length of solar capacity factor data
                T = len(s)

                # Set constants based on the type of hydrogen production: continuous or flexible
                if production_type == "continuous":
                    # Set constant hourly H2 demand for continuous production
                    D_H2 = [7500] * T # kg H2/h
                    hourly_demand_H2 = 7500 # kg H2/h
                    delta_H2 = 1 # Plant operates at nominal capacity
                    ASU_S_costs = 0 # No additional costs for Air Separation Unit (ASU) and NH3 synthesis loop for continuous production
                elif production_type == "flexible":
                    # Set variable hourly H2 demand for flexible production
                    D_H2_h = 7500
                    hourly_demand_H2 = D_H2_h
                    delta_H2 = 0.5 # Plant operates at a minimum of 50% of nominal capacity
                    ASU_S_costs = 300000000 * 1/delta_H2 # Additional costs for replacement ASU and NH3 synthesis loop for flexible production

                # Define additional variables for LCOH calculation
                hours_year = 8760 # Number of hours in a year
                lifetime_H2_demand = lifetime_plant * hourly_demand_H2 * hours_year
                annual_H2_demand = hourly_demand_H2 * hours_year

                # Set initial conditions for energy storage
                S_E_B_0 = 0 # Initial energy stored in batteries, assumed to be zero
                S_H2_ST_0 = 0 # Initial hydrogen stored in tanks, assumed to be zero

                # Define minimum installed capacities for various components of the EHPS (in respective units)
                P_PV_min = 10 # PV in MW
                P_WT_min = 10 # Wind Turbines (WT) in MW
                P_EL_min = hourly_demand_H2 * energy_demand_electrolyzers # Electrolyzers in MW
                P_B_min = 10 # Batteries in MWh
                P_CP_min = 10 # Compressor in kg H2/h
                P_ST_min = 100 # Hydrogen storage tanks in kg H2

                # Define maximum installed capacities for various components of the EHPS (in respective units)
                P_PV_max = 5e4 # PV in MW
                P_WT_max = 5e4 # Wind Turbines (WT) in MW
                P_EL_max = 5e5 # Electrolyzers in MW
                P_B_max = 5e4 # Batteries in MWh
                P_CP_max = 5e5 # Compressor in kg H2/h
                P_ST_max = 1e6 # Hydrogen storage tanks in kg H2
                
                # Electrolyzer
                # Assume 'electrolyzer' is defined elsewhere in the code.
                # Set energy demand for electrolyzers based on the type and scenarios
                if electrolyzer == 'ALK':
                    # For Alkaline electrolyzers, set energy demand based on the scenario
                    if 'AVG' in scenarios:
                        energy_demand_electrolyzers = 0.0503 # MWh/kg H2 for average scenario
                    elif 'OPT' in scenarios:
                        energy_demand_electrolyzers = 0.0473 # MWh/kg H2 for optimistic scenario
                    elif 'PES' in scenarios:
                        energy_demand_electrolyzers = 0.0521 # MWh/kg H2 for pessimistic scenario
                elif electrolyzer == 'SOE':
                    # For Solid Oxide Electrolyzers, use a fixed value
                    energy_demand_electrolyzers = 0.0422 # MWh/kg H2
                elif electrolyzer == 'ML':
                    # For Membraneless Electrolyzers, use a fixed value
                    energy_demand_electrolyzers = 0.0693 # MWh/kg H2
                    
                eta_EL = 1/energy_demand_electrolyzers
                
                # Create model
                m = Model()
                
                # if variable production == True:
                if production_type == "flexible":
                    D_H2 = m.addVars(T, lb=0, name = "D_H2")
                # Imported grid electricity
                M_E = m.addVars(T, lb=0, name="M_E") # imported grid electricity in MWh
                # Installed capacity components
                P_PV = m.addVar(lb=0, name="P_PV") # installed capacity solar PV in MW
                P_WT = m.addVar(lb=0, name="P_WT") # installed capacity wind turbines in MW
                P_EL = m.addVar(lb=P_EL_min, name="P_EL") # installed capacity electrolyzers in MW
                P_B = m.addVar(lb=0, name="P_B") # installed capacity batteries in MWh
                P_CP = m.addVar(lb=0, name="P_CP") # installed capacity compressor in kg H2 compressed/h
                P_ST = m.addVar(lb=0, name="P_ST") # installed capacity high-pressure H2 storage tanks in kg H2 capacity
                # Technology selection
                # Energy carrier input
                U_E_EL = m.addVars(T, lb=0, name="U_E_EL") # input electricity electrolyzers in MWh
                U_E_B = m.addVars(T, lb=0, name="U_E_B") # input electricity in batteries in MWh
                U_E_CP = m.addVars(T, lb=0, name="U_E_CP") # input electricity in compressor in MWh
                U_H2_CP = m.addVars(T, lb=0, name="U_H2_CP") # input hydrogen in compressor in kg H2/h
                U_H2_ST = m.addVars(T, lb=0, name="U_H2_ST") # input hydrogen in storage tanks in kg H2/h
                # Energy carrier output
                V_E_PV = m.addVars(T, lb=0, name="V_E_PV") # output electricity from solar PV in MW
                V_E_WT = m.addVars(T, lb=0, name="V_E_WT") # output electricity from wind turbines in MW
                V_E_B = m.addVars(T, lb=0, name="V_E_B") # output electricity from batteries in MWh
                V_H2_EL = m.addVars(T, lb=0, name="V_H2_EL") # output hydrogen electrolyzers in kg H2/h
                V_H2_CP = m.addVars(T, lb=0, name="V_H2_CP") # output hydrogen compressors in kg H2/h
                V_H2_ST = m.addVars(T, lb=0, name="V_H2_ST") # output hydrogen storage tanks in kg H2/h
                # Energy stored
                S_E_B = m.addVars(T, lb=0, name="S_E_B") # stored electricity in batteries in MWh
                S_H2_ST = m.addVars(T, lb=0, name="S_H2_ST") # stored hydrogen in storage tanks in kg H2   
               
                # imported and exported electricity
                m.addConstrs(M_E[t] >= 0 for t in range(T)) # imported electricity non negative
                # emission constraints
                m.addConstr(quicksum(M_E[t] * gamma_E for t in range(T)) <= quicksum(D_H2[t] * epsilon_H2 for t in range(T)))
                # renewables generation
                m.addConstrs(V_E_PV[t] == s[t] * eta_PV * P_PV for t in range(T)) # hourly output of PV = solar CF * installed capacity PV
                m.addConstrs(V_E_WT[t] == w[t] * eta_WT * P_WT for t in range(T)) # hourly output of WIND turbines = wind CF * installed capacity WT
                # batteries
                m.addConstr(S_E_B[0] == S_E_B_0) # initial stored energy in batteries = 0 MWh
                m.addConstr(V_E_B[0] == S_E_B_0) # initial stored energy in batteries = 0 MWh
                m.addConstr(S_E_B[T-1] == S_E_B[0]) # periodicity contraint: stored energy at beginning and at the end of the time horizon equal
                m.addConstrs(U_E_B[t] <= P_B/tau_c_B for t in range(T)) # imported energy <= installed capacity batteries
                m.addConstrs(U_E_B[t] >= 0 for t in range(T)) # input energy battery non negative
                m.addConstrs(S_E_B[t] == (1-lambda_B) * S_E_B[t-1] + eta_c_B * U_E_B[t] - (V_E_B[t]/eta_d_B) for t in range(1, T)) # stored energy = stored energy previous iteration + new imported energy - output energy. 
                m.addConstrs(S_E_B[t] <= P_B for t in range(T)) # stored energy <= installed capacity batteries
                m.addConstrs(S_E_B[t] >= 0 for t in range(T)) # stored energy battery non negative
                m.addConstrs(V_E_B[t] <= P_B/tau_d_B for t in range(T)) # output energy <= installed capacity batteries
                m.addConstrs(V_E_B[t] >= 0 for t in range(T)) # output energy battery non negative
                # electrolyzer
                m.addConstrs(U_E_EL[t] <= P_EL for t in range(T)) # input electorlyzer <= installed capacity electrolyzer
                m.addConstrs(U_E_EL[t] >= 0 for t in range(T)) # input electricity electrolyzer non negative
                m.addConstrs(V_H2_EL[t] == eta_EL * U_E_EL[t] for t in range(T)) # output electrolyzer = performance * input energy
                # compressors
                m.addConstrs(U_E_CP[t] * eta_CP <= P_CP for t in range(T)) # input compressors <= performance * installed capacity compressors
                m.addConstrs(U_E_CP[t] * eta_CP >= 0 for t in range(T)) # input electricity compressors non negative
                m.addConstrs(U_H2_CP[t] == V_H2_CP[t] for t in range(T)) # output compressors = performance * input energy
                m.addConstrs(U_H2_CP[t] >= 0 for t in range(T)) # input H2 compressors non negative
                m.addConstrs(V_H2_CP[t] == eta_CP * U_E_CP[t] for t in range(T)) # output compressors = performance * input energy
                m.addConstrs(V_H2_EL[t] >= U_H2_CP[t] for t in range(T)) # compressor H2 input <= electrolyzer H2 output
                # high-pressure tanks for H2 storage
                m.addConstr(S_H2_ST[0] == S_H2_ST_0) # initial stored H2 in tanks = 0 kg H2
                m.addConstr(V_H2_ST[0] == S_H2_ST_0) # periodicity contraint: stored H2 at beginning and at the end of the time horizon equal
                m.addConstr(S_H2_ST[T-1] == S_H2_ST[0]) # periodicity contraint: stored H2 at beginning and at the end of the time horizon equal                
                m.addConstrs(U_H2_ST[t] == V_H2_CP[t] for t in range(T)) # output H2 compressor = input H2 storage tanks
                m.addConstrs(S_H2_ST[t] == S_H2_ST[t-1] + U_H2_ST[t] - V_H2_ST[t] for t in range(1, T)) # stored energy = stored energy previous iteration + new imported energy - output energy. 
                m.addConstrs(S_H2_ST[t] <= P_ST for t in range(T)) # stored H2 <= installed capacity tanks
                m.addConstrs(S_H2_ST[t] >= 0 for t in range(T)) # stored H2 tanks non negative
                m.addConstrs(V_H2_ST[t] <= P_ST for t in range(T)) # output H2 <= installed capacity tanks
                m.addConstrs(V_H2_ST[t] >= 0 for t in range(T)) # output H2 tanks non negative
                # energy balance
                m.addConstrs(M_E[t] + V_E_PV[t] + V_E_WT[t] - U_E_B[t] + V_E_B[t] >= U_E_EL[t] + U_E_CP[t] for t in range(T)) # imported energy + output PV and WT - input energy batteries + output batteries >= input electrolyzer + input compressor
                m.addConstrs(V_H2_EL[t] - U_H2_ST[t] + V_H2_ST[t] == D_H2[t] for t in range(T)) # output electrolyzer >= hourly demand of H2
                # if flexible
                if production_type == "flexible":
                    m.addConstrs(D_H2[t] >= 7500 * delta_H2 for t in range(T))
                    m.addConstrs(D_H2[t] <= 7500 * 1/delta_H2 for t in range(T))
                    m.addConstr(quicksum(D_H2[t] for t in range (T)) >= annual_H2_demand)

                CAPEX_PV = c_PV * P_PV 
                OPEX_PV = P_PV * OM_PV
                A_TOT_PV = P_PV * A_PV
                COST_LAND_PV = land_price * A_TOT_PV 
                CAPEX_WT = c_WT * P_WT
                OPEX_WT = P_WT * OM_WT
                A_TOT_WT = P_WT * A_WT
                COST_LAND_WT = land_price * A_TOT_WT 
                CAPEX_EL = c_EL * P_EL
                OPEX_EL = c_EL * P_EL * OM_EL
                CAPEX_B = c_B * P_B
                OPEX_B = c_B * P_B * OM_B
                OPEX_E = p_E * (1/eta_grid) * quicksum(M_E[t] for t in range(T))
                OPEX_ETS = 0 # p_ETS * gamma_E * quicksum(M_E[t] for t in range(T))
                CAPEX_CP = c_CP * P_CP
                OPEX_CP = c_CP * P_CP * OM_CP
                CAPEX_ST = c_ST * P_ST
                CAPEX_GRID_CONNECTION = T_HVAC + (c_HVAC_wires * distance)

                z_cost = (ASU_S_costs + OPEX_ETS + CAPEX_GRID_CONNECTION + T_LVAC + CAPEX_PV + OPEX_PV + CAPEX_WT + OPEX_WT + CAPEX_EL + OPEX_EL + CAPEX_B + OPEX_B + OPEX_E + CAPEX_CP + OPEX_CP + CAPEX_ST + COST_LAND_PV + COST_LAND_WT + COST_RETROFIT)

                m.setObjective(z_cost, GRB.MINIMIZE)    

                # m.setParam('Presolve', 1)
                # m.setParam('Symmetry', 1)
                # m.setParam('Heuristics', 1)
                # m.setParam('Cuts', 2)
                # m.setParam('MIPFocus', 1)

                # Optimize model
                m.optimize()

                # Create empty lists
                PV_output = []
                WT_output = []
                imported_electricity = []
                exported_electricity = []
                electrolyzer_electricity_input = []
                electrolyzer_H2_output = []
                battery_electricity_input = []
                battery_electricity_stored = []
                battery_electricity_output = []
                compressor_electricity_input = []
                compressor_H2_input = []
                compressor_H2_output = []
                tanks_H2_input = []
                tanks_H2_stored = []
                tanks_H2_output = []    

                # Iterate over time steps and append values to lists
                for t in range(T):
                    PV_output.append(V_E_PV[t].x)
                    WT_output.append(V_E_WT[t].x)
                    imported_electricity.append(M_E[t].x)
                    electrolyzer_electricity_input.append(U_E_EL[t].x)
                    electrolyzer_H2_output.append(V_H2_EL[t].x)
                    battery_electricity_input.append(U_E_B[t].x)
                    battery_electricity_stored.append(S_E_B[t].x)
                    battery_electricity_output.append(V_E_B[t].x)
                    compressor_electricity_input.append(U_E_CP[t].x)
                    compressor_H2_input.append(U_H2_CP[t].x)
                    compressor_H2_output.append(V_H2_CP[t].x)
                    tanks_H2_input.append(U_H2_ST[t].x)
                    tanks_H2_stored.append(S_H2_ST[t].x)
                    tanks_H2_output.append(V_H2_ST[t].x)           


                # Calculate sums of the entries in each list
                PV_output_sum = sum(PV_output) 
                WT_output_sum = sum(WT_output) 
                imported_electricity_sum = sum(imported_electricity) 
                exported_electricity_sum = sum(exported_electricity) 
                electrolyzer_electricity_input_sum = sum(electrolyzer_electricity_input) 
                electrolyzer_H2_output_sum = sum(electrolyzer_H2_output) 
                battery_electricity_input_sum = sum(battery_electricity_input) 
                battery_electricity_stored_sum = sum(battery_electricity_stored) 
                battery_electricity_output_sum = sum(battery_electricity_output) 
                compressor_electricity_input_sum = sum(compressor_electricity_input) 
                compressor_H2_input_sum = sum(compressor_H2_input) 
                compressor_H2_output_sum = sum(compressor_H2_output) 
                tanks_H2_input_sum = sum(tanks_H2_input) 
                tanks_H2_stored_sum = sum(tanks_H2_stored) 
                tanks_H2_output_sum = sum(tanks_H2_output) 

                LCOH = z_cost.getValue()/lifetime_H2_discount
                carbon_emissions_H2 = (imported_electricity_sum * gamma_E)/(hourly_demand_H2 * hours_year)
                total_energy_entering_the_system = imported_electricity_sum + PV_output_sum + WT_output_sum
                renewable_energy_entering_the_system = PV_output_sum + WT_output_sum
                total_energy_consumed_plant = electrolyzer_electricity_input_sum + battery_electricity_input_sum - battery_electricity_output_sum + compressor_electricity_input_sum
                En_frac_grid = imported_electricity_sum/(total_energy_consumed_plant)
                ratio_energy_compressor_total = compressor_electricity_input_sum/total_energy_consumed_plant
                ratio_energy_electrolyzer_total = electrolyzer_electricity_input_sum/total_energy_consumed_plant
                ratio_input_energy_battery_total = battery_electricity_input_sum/total_energy_consumed_plant
                ratio_output_energy_battery_total = battery_electricity_output_sum/total_energy_consumed_plant

                ratio_energy_consumed_energy_input = total_energy_consumed_plant/total_energy_entering_the_system
                
                if renewable_energy_entering_the_system == 0:
                    curtailment = 0
                else:
                    curtailment = (1 - ((total_energy_consumed_plant-imported_electricity_sum)/renewable_energy_entering_the_system))

                # Append results to dataframe for region
                data_design = {
                    'LCOH': LCOH,
                    'z_cost': z_cost.getValue(),
                    'carbon_emissions_H2': carbon_emissions_H2,
                    'P_PV': P_PV.x,
                    'P_WT': P_WT.x,
                    'P_EL': P_EL.x,
                    'P_B': P_B.x,
                    'P_CP': P_CP.x,
                    'P_ST': P_ST.x,
                    'PV_output_sum': PV_output_sum,
                    'WT_output_sum': WT_output_sum,
                    'imported_electricity_sum': imported_electricity_sum,
                    'exported_electricity_sum': exported_electricity_sum,
                    'En_frac_grid': En_frac_grid,
                    'Total_energy_consumed_plant': total_energy_consumed_plant,
                    'Total_energy_entering_the_system': total_energy_entering_the_system,
                    'Curtailment': curtailment,
                    'Ratio_energy_consumed_energy_input': ratio_energy_consumed_energy_input,
                    'electrolyzer_electricity_input_sum': electrolyzer_electricity_input_sum,
                    'electrolyzer_H2_output_sum': electrolyzer_H2_output_sum,
                    'battery_electricity_input_sum': battery_electricity_input_sum ,
                    'battery_electricity_stored_sum': battery_electricity_stored_sum,
                    'battery_electricity_output_sum': battery_electricity_output_sum,
                    'compressor_H2_input_sum': compressor_H2_input_sum,
                    'compressor_H2_output_sum': compressor_H2_output_sum, 
                    'tanks_H2_input_sum': tanks_H2_input_sum,
                    'tanks_H2_stored_sum': tanks_H2_stored_sum,
                    'tanks_H2_output_sum': tanks_H2_output_sum,  
                    'c_PV': c_PV,
                    'OM_PV': OM_PV,
                    'c_WT': c_WT,
                    'OM_WT': OM_WT, 
                    'p_E': p_E,
                    'gamma_E': gamma_E,
                    'epsilon_H2': epsilon_H2,
                    'lifetime_plant': lifetime_plant,
                    'c_EL': c_EL,
                    'OM_EL': OM_EL,
                    'c_B': c_B,
                    'OM_B': OM_B,
                    'c_CP': c_CP,
                    'OM_CP': OM_CP,
                    'c_ST': c_ST,
                    'T_HVAC': T_HVAC,
                    'T_LVAC': T_LVAC,
                    'c_HVAC_wires': c_HVAC_wires,
                    'Distance': distance,
                    #'p_ETS': p_ETS,
                    'COST_RETROFIT': COST_RETROFIT
                }

                data_operation = {
                    'PV_output': PV_output,
                    'WT_output': WT_output, 
                    'imported_electricity': imported_electricity,
                    'electrolyzer_electricity_input': electrolyzer_electricity_input,
                    'electrolyzer_H2_output': electrolyzer_H2_output, 
                    'battery_electricity_input': battery_electricity_input,
                    'battery_electricity_stored': battery_electricity_stored,
                    'battery_electricity_output': battery_electricity_output,                   
                    'compressor_electricity_input': compressor_electricity_input,
                    'compressor_H2_input': compressor_H2_input,
                    'compressor_H2_output': compressor_H2_output,
                    'tanks_H2_input': tanks_H2_input,
                    'tanks_H2_stored': tanks_H2_stored,
                    'tanks_H2_output': tanks_H2_output
                }

                df_design = df_design.append(pd.DataFrame(data_design, index=[0]), ignore_index=True)

                df_design['CAPEX_PV'] = df_design['c_PV'] * df_design['P_PV']
                df_design['OPEX_PV'] = df_design['P_PV'] * OM_PV
                df_design['CAPEX_WT'] = df_design['c_WT'] * df_design['P_WT']
                df_design['OPEX_WT'] = df_design['P_WT'] * OM_WT
                df_design['CAPEX_EL'] = df_design['c_EL'] * df_design['P_EL']
                df_design['OPEX_EL'] = df_design['c_EL'] * df_design['P_EL'] * OM_EL
                df_design['CAPEX_B'] = df_design['c_B'] * df_design['P_B']
                df_design['OPEX_B'] = df_design['c_B'] * df_design['P_B'] * OM_B
                df_design['OPEX_E'] = df_design['imported_electricity_sum'] * (1/eta_grid) * df_design['p_E']
                df_design['CAPEX_CP'] = df_design['c_CP'] * df_design['P_CP']
                df_design['OPEX_CP'] = df_design['c_CP'] * df_design['P_CP'] * OM_CP
                df_design['CAPEX_ST'] = df_design['c_ST'] * df_design['P_ST']
                df_design['Cost_land_PV'] = df_design['P_PV'] * A_PV * land_price 
                df_design['Cost_land_WT'] = df_design['P_WT'] * A_WT * land_price
                df_design['COST_RETROFIT'] = COST_RETROFIT
                df_design['CAPEX_GRID_CONNECTION'] = T_HVAC + (c_HVAC_wires * distance)   
                df_design['CAPEX_T_LVAC'] = T_LVAC
                #df_design['OPEX_ETS'] = df_design['imported_electricity_sum'] * (gamma_E) * df_design['p_ETS']
                df_design['Total_cost'] = df_design['z_cost']
                df_design['A_TOT_PV'] = df_design['P_PV'] * A_PV
                df_design['A_TOT_WT'] = df_design['P_WT'] * A_WT

                df_operation = df_operation.append(pd.DataFrame(data_operation), ignore_index=True)
                
                # Generate the parameter part of the filename
                parameters_str = '_'.join(parameters) if parameters else ''

                # Generate the scenario part of the filename
                scenario_str = scenario if parameters else 'AVG'

                # Store results in dataframe for region
                df_name_design = f"design_{region}_{epsilon_H2_str}_{scenario_str}_{parameters_str}".rstrip('_')
                globals()[df_name_design] = df_design

                # Store results in dataframe for operation
                df_name_operation = f"operation_{region}_{epsilon_H2_str}_{scenario_str}_{parameters_str}".rstrip('_')
                globals()[df_name_operation] = df_operation

                # Ensure the 'results' directory exists
                os.makedirs('results', exist_ok=True)

                # Refer to the variables using `globals()` and save to CSV
                globals()[df_name_design].to_csv(f'results/{df_name_design}.csv')
                globals()[df_name_operation].to_csv(f'results/{df_name_operation}.csv')

---